In [ ]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *

spark = sphynx.get_spark(executor_count=2, app_name='pdu_syp', node_spec=NODE_LARGE)

In [124]:
sphynx.stop()

Stopping Spark session...
Destroying Spark cluster...
Done!
You can safely ignore the warning message if there is any.


### eSports PU & SALES 구하는 조건

In [26]:
pcs1_cash = pre_load_data_mart(start_date='2020-06-17', end_date='2020-07-02', device='pc', table_name='cash_mtx')

In [27]:
pcs1_cash.where("event_type = 'esports' and status = 'Succeeded'").groupBy("event_name").agg(countDistinct("account_id")).show()

+-----------+--------------------------+
| event_name|count(DISTINCT account_id)|
+-----------+--------------------------+
|pcs_sales_1|                    240295|
+-----------+--------------------------+



In [29]:
pcs1_cash.where("event_type = 'esports' and status = 'Succeeded'").groupBy("event_name").agg(sum("price_"), sum('ingame_price')).show()

+-----------+-----------+-----------------+
| event_name|sum(price_)|sum(ingame_price)|
+-----------+-----------+-----------------+
|pcs_sales_1| 2711918.49|       2711918.49|
+-----------+-----------+-----------------+



In [30]:
pcs3_gcoin = pre_load_data_mart(start_date='2020-10-28', end_date='2020-11-19', device='pc', table_name='gcoin_use')

In [33]:
pcs3_gcoin.where("event_type = 'esports'").groupBy("event_name").agg(sum("price")).show()

+-----------+----------+
| event_name|sum(price)|
+-----------+----------+
|pcs_sales_3| 234061120|
+-----------+----------+



In [3]:
cash = load_data_mart(device='pc', start_date='2021-10-04', end_date='2021-10-10', table_name='cash_mtx')

In [4]:
gcoin = load_data_mart(device='pc', start_date='2021-10-04', end_date='2021-10-10', table_name='gcoin_use')

In [9]:
cash.groupBy('date').sum('price_', 'ingame_price').orderBy('date').show()

+----------+-----------+-----------------+
|      date|sum(price_)|sum(ingame_price)|
+----------+-----------+-----------------+
|2021-10-04|  480187.16|        480187.16|
|2021-10-05|  445938.34|        445938.34|
|2021-10-06| 2957925.66|       2957925.66|
|2021-10-07| 1998655.99|       1998655.99|
|2021-10-08| 1389152.35|       1389152.35|
|2021-10-09| 1237633.30|       1237633.30|
|2021-10-10| 1150840.68|       1150840.68|
+----------+-----------+-----------------+



In [16]:
gcoin.groupBy('date').sum('free_use', 'paid_use', 'price').orderBy('date').show()

+----------+-------------+-------------+----------+
|      date|sum(free_use)|sum(paid_use)|sum(price)|
+----------+-------------+-------------+----------+
|2021-10-04|     29048710|     46457120|  74195050|
|2021-10-05|     32186380|     42495970|  73323380|
|2021-10-06|     77771950|    330871780| 407601440|
|2021-10-07|     64016030|    216198850| 278781290|
|2021-10-08|     48176380|    147526740| 194522980|
|2021-10-09|     67472790|    130231080| 196183330|
|2021-10-10|     75282140|    119551120| 193197790|
+----------+-------------+-------------+----------+



In [34]:
cash.count()

460408

In [19]:
gcoin.where(''' lower(product_name) like '%bundle%' ''').groupBy('date').sum('price').orderBy("date").show()

+----------+----------+
|      date|sum(price)|
+----------+----------+
|2021-10-04|  10509230|
|2021-10-05|   9894820|
|2021-10-06|   8327570|
|2021-10-07|  10078080|
|2021-10-08|   9492510|
|2021-10-09|  12735440|
|2021-10-10|  15316250|
+----------+----------+



In [24]:
total = gcoin.groupBy('date').agg(sum('price').alias('total_price'))
bundle = gcoin.where(''' lower(product_name) like '%bundle%' ''').groupBy('date').agg(sum('price').alias('bundle_sum'))

total.join(bundle, total.date == bundle.date).withColumn('ratio', bundle.bundle_sum/total.total_price ).orderBy(total.date).show()

+----------+-----------+----------+----------+--------------------+
|      date|total_price|      date|bundle_sum|               ratio|
+----------+-----------+----------+----------+--------------------+
|2021-10-04|   74195050|2021-10-04|  10509230|  0.1416432767415077|
|2021-10-05|   73323380|2021-10-05|   9894820| 0.13494767971689248|
|2021-10-06|  407601440|2021-10-06|   8327570|0.020430668743466655|
|2021-10-07|  278781290|2021-10-07|  10078080|   0.036150489152267|
|2021-10-08|  194522980|2021-10-08|   9492510| 0.04879891311556095|
|2021-10-09|  196183330|2021-10-09|  12735440| 0.06491601503552825|
|2021-10-10|  193197790|2021-10-10|  15316250| 0.07927756316467181|
+----------+-----------+----------+----------+--------------------+



In [49]:
total.join(bundle, total.date == bundle.date).withColumn('percentage', round(bundle.bundle_sum/total.total_price*100, 2) ).orderBy(total.date).show()

+----------+-----------+----------+----------+----------+
|      date|total_price|      date|bundle_sum|percentage|
+----------+-----------+----------+----------+----------+
|2021-10-04|   74195050|2021-10-04|  10509230|     14.16|
|2021-10-05|   73323380|2021-10-05|   9894820|     13.49|
|2021-10-06|  407601440|2021-10-06|   8327570|      2.04|
|2021-10-07|  278781290|2021-10-07|  10078080|      3.62|
|2021-10-08|  194522980|2021-10-08|   9492510|      4.88|
|2021-10-09|  196183330|2021-10-09|  12735440|      6.49|
|2021-10-10|  193197790|2021-10-10|  15316250|      7.93|
+----------+-----------+----------+----------+----------+



- blackpink mega bundle: itemdesc.13000318
    - 9900 gcoin, $100 

In [41]:
gcoin.where(" product_id ==  'itemdesc.13000318' ").groupBy("date").agg(count("*"), sum("price")).orderBy('date').show()

+----------+--------+----------+
|      date|count(1)|sum(price)|
+----------+--------+----------+
|2021-10-04|     119|   1178100|
|2021-10-05|     118|   1168200|
|2021-10-06|     101|    999900|
|2021-10-07|     115|   1138500|
|2021-10-08|      92|    910800|
|2021-10-09|     119|   1178100|
|2021-10-10|     132|   1306800|
+----------+--------+----------+



In [44]:
# Blackpink BUNDLE
gcoin.where(" product_id ==  'itemdesc.13000318' ").groupBy("date", "platform").agg(count("*"), sum("price")).orderBy('date', 'platform').show()

+----------+--------+--------+----------+
|      date|platform|count(1)|sum(price)|
+----------+--------+--------+----------+
|2021-10-04|   KAKAO|       9|     89100|
|2021-10-04|   STEAM|     110|   1089000|
|2021-10-05|   KAKAO|      13|    128700|
|2021-10-05|   STEAM|     105|   1039500|
|2021-10-06|   KAKAO|      13|    128700|
|2021-10-06|   STEAM|      88|    871200|
|2021-10-07|   KAKAO|       9|     89100|
|2021-10-07|   STEAM|     106|   1049400|
|2021-10-08|   KAKAO|       9|     89100|
|2021-10-08|   STEAM|      83|    821700|
|2021-10-09|   KAKAO|      12|    118800|
|2021-10-09|   STEAM|     107|   1059300|
|2021-10-10|   KAKAO|      11|    108900|
|2021-10-10|   STEAM|     121|   1197900|
+----------+--------+--------+----------+



In [47]:
# Total BUNDLE
gcoin.where(""" lower(product_name) like '%bundle%' """).groupBy("date", "platform").agg(count('*').alias('cnt'), sum('price').alias('sum_price')).orderBy('date', 'platform').show()

+----------+--------+----+---------+
|      date|platform| cnt|sum_price|
+----------+--------+----+---------+
|2021-10-04|   KAKAO| 595|  1199210|
|2021-10-04|   STEAM|5033|  9310020|
|2021-10-05|   KAKAO| 455|  1002420|
|2021-10-05|   STEAM|4842|  8892400|
|2021-10-06|   KAKAO| 339|   742200|
|2021-10-06|  MAILRU|   1|      990|
|2021-10-06|   STEAM|4336|  7584380|
|2021-10-07|   KAKAO| 358|   742630|
|2021-10-07|   STEAM|5337|  9335450|
|2021-10-08|   KAKAO| 517|  1072150|
|2021-10-08|   STEAM|4817|  8420360|
|2021-10-09|   KAKAO| 503|  1051460|
|2021-10-09|  MAILRU|   1|      560|
|2021-10-09|   STEAM|7244| 11683420|
|2021-10-10|   KAKAO| 598|  1256160|
|2021-10-10|  MAILRU|   1|     1590|
|2021-10-10|   STEAM|8685| 14058500|
+----------+--------+----+---------+



#### 2주간 추이 
- 21.09.27 ~ 21.10.10

In [72]:
user = load_data_mart(device='pc', start_date='2021-09-27', end_date='2021-10-10', table_name='user_master')
user = user.where(col('lastlogindate') >= '2021-09-27')

In [74]:
gcoin = load_data_mart(device='pc', start_date='2021-09-27', end_date='2021-10-10', table_name='gcoin_use')

In [75]:
print('The length of the user_master', user.count())

The length of the user_master 47890492


In [80]:
user.groupBy('date', 'platform').agg(countDistinct('accountid').alias('AU')).orderBy('date', 'platform').show()

+----------+--------+-------+
|      date|platform|     AU|
+----------+--------+-------+
|2021-09-27|   KAKAO|  96944|
|2021-09-27|  MAILRU|   1606|
|2021-09-27|   STEAM|1166259|
|2021-09-28|   KAKAO| 144150|
|2021-09-28|  MAILRU|   2180|
|2021-09-28|   STEAM|1690297|
|2021-09-29|   KAKAO| 179386|
|2021-09-29|  MAILRU|   2557|
|2021-09-29|   STEAM|2043186|
|2021-09-30|   KAKAO| 207068|
|2021-09-30|  MAILRU|   2835|
|2021-09-30|   STEAM|2391340|
|2021-10-01|   KAKAO| 241013|
|2021-10-01|  MAILRU|   3112|
|2021-10-01|   STEAM|2745819|
|2021-10-02|   KAKAO| 281921|
|2021-10-02|  MAILRU|   3436|
|2021-10-02|   STEAM|3042003|
|2021-10-03|   KAKAO| 316433|
|2021-10-03|  MAILRU|   3710|
+----------+--------+-------+
only showing top 20 rows



In [85]:
gcoin.groupBy('date', 'platform').agg(countDistinct('account_id').alias('PU'), sum('price').alias('gcoin_sum')).orderBy('date', 'platform').show()

+----------+--------+------+---------+
|      date|platform|    PU|gcoin_sum|
+----------+--------+------+---------+
|2021-09-27|   KAKAO|  1932|  4148130|
|2021-09-27|  MAILRU|    13|     9660|
|2021-09-27|   STEAM| 22246| 53744060|
|2021-09-28|   KAKAO|  1985|  4548370|
|2021-09-28|  MAILRU|     9|    12290|
|2021-09-28|   STEAM| 32455| 60633980|
|2021-09-29|   KAKAO|  7447|  9670350|
|2021-09-29|  MAILRU|    29|    23780|
|2021-09-29|   STEAM|105587|128246410|
|2021-09-30|   KAKAO|  4233|  6655730|
|2021-09-30|  MAILRU|    20|    46300|
|2021-09-30|   STEAM| 75866| 82555890|
|2021-10-01|   KAKAO|  4291|  7450650|
|2021-10-01|  MAILRU|    12|     9790|
|2021-10-01|   STEAM| 79679| 75406580|
|2021-10-02|   KAKAO|  4285|  7246160|
|2021-10-02|  MAILRU|    11|     8870|
|2021-10-02|   STEAM| 84533| 70624090|
|2021-10-03|   KAKAO|  3979|  6533420|
|2021-10-03|  MAILRU|    12|     3870|
+----------+--------+------+---------+
only showing top 20 rows



In [115]:
au = user.groupBy('date', 'platform').agg(countDistinct('accountid').alias('au'))
pu = gcoin.groupBy('date', 'platform').agg(countDistinct('account_id').alias('pu'), sum('price').alias('gcoin_sum'))
pur = au.join(pu, ["date", "platform"], 'left').withColumn('pur', round(pu.pu/au.au, 4))

In [116]:
pur.orderBy('date', 'platform').show()

+----------+--------+-------+------+---------+------+
|      date|platform|     au|    pu|gcoin_sum|   pur|
+----------+--------+-------+------+---------+------+
|2021-09-27|   KAKAO|  96944|  1932|  4148130|0.0199|
|2021-09-27|  MAILRU|   1606|    13|     9660|0.0081|
|2021-09-27|   STEAM|1166259| 22246| 53744060|0.0191|
|2021-09-28|   KAKAO| 144150|  1985|  4548370|0.0138|
|2021-09-28|  MAILRU|   2180|     9|    12290|0.0041|
|2021-09-28|   STEAM|1690297| 32455| 60633980|0.0192|
|2021-09-29|   KAKAO| 179386|  7447|  9670350|0.0415|
|2021-09-29|  MAILRU|   2557|    29|    23780|0.0113|
|2021-09-29|   STEAM|2043186|105587|128246410|0.0517|
|2021-09-30|   KAKAO| 207068|  4233|  6655730|0.0204|
|2021-09-30|  MAILRU|   2835|    20|    46300|0.0071|
|2021-09-30|   STEAM|2391340| 75866| 82555890|0.0317|
|2021-10-01|   KAKAO| 241013|  4291|  7450650|0.0178|
|2021-10-01|  MAILRU|   3112|    12|     9790|0.0039|
|2021-10-01|   STEAM|2745819| 79679| 75406580| 0.029|
|2021-10-02|   KAKAO| 281921